<img align="right" src="../img/auvisus.svg" width="100" >


# DEtection TRansformer Network

---

## 1. Import Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import colorsys
import random
import pickle

import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img

from matplotlib import patches
import matplotlib.pyplot as plt

from detr_models.detr.model import DETR
from detr_models.detr.config import DefaultDETRConfig
from detr_models.backbone.backbone import Backbone
from detr_models.detr.utils import create_positional_encodings, get_image_information
from detr_models.data_feeder.coco_feeder import COCOFeeder

In [ ]:
def add_mask_to_image(image, mask, color, alpha=0.5):
    for c in range(3):
        image[:, :, c] = np.where(
            mask == 1,
            image[:, :, c] * (1 - alpha) + alpha * color[c] * 255,
            image[:, :, c],
        )
    return image

def apply_bbox(ax, bbox, color):
    x1, y1, w, h = bbox
    p = patches.Rectangle(
        (x1, y1),
        w,
        h,
        linewidth=2,
        alpha=0.7,
        linestyle="dashed",
        edgecolor=color,
        facecolor="none",
    )

    ax.add_patch(p)


---

## 2. Specify required paths

In [ ]:
# Specify model path
path = input(prompt='Please specify the storage path:\n')

---


## 3. Load Model

In [ ]:
model = tf.keras.models.load_model(os.path.join(path, "detr_model"))
model.summary()

---

## 3. Plot Training Loss

In [ ]:
with open(os.path.join(path, "detr_loss.txt"),'rb') as detr_loss:
    training_loss = pickle.load(detr_loss)
    training_loss = np.array(training_loss)

detr_loss = training_loss[:,0]
score_loss  = training_loss[:,1]
bbox_loss = training_loss[:,2]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.set_title("Training Loss DETR - Epochs {}".format(len(detr_loss)))
ax.plot(detr_loss, label="Total DETR Loss")
ax.plot(score_loss, label="Class Loss")
ax.plot(bbox_loss, label="Bounding Box Loss")

ax.legend()
plt.show()

---

## 4. Load Data

In [ ]:
# Specify storage path
# Points to /tests/data by default, but you should include your own data path to verify the data feeder
storage_path = "../tests/data"

# Specify batch size
batch_size = 1


# Specify Whether Model contains Segmentation Head
train_masks = True

In [ ]:
config = DefaultDETRConfig()

training_config = {
        "storage_path": storage_path,
        "batch_size": batch_size,
        "train_masks": train_masks,
    }

In [ ]:
data_feeder = COCOFeeder(
        storage_path=training_config["storage_path"],
        batch_size=training_config["batch_size"],
        num_queries=config.num_queries,
        num_classes=config.num_classes,
        image_width=config.image_width,
        image_height=config.image_height,
    )

In [ ]:
input_data = next(data_feeder(False))

batch_images = input_data[0]
batch_cls = input_data[1]
batch_bboxs = input_data[2]
obj_indices = input_data[3]
batch_masks = input_data[4]

print("Batch Images: {}".format(batch_images.shape))
print("Batch Target Class Labels: {}".format(batch_cls.shape))
print("Batch Target Bounding Boxes: {}".format(batch_bboxs.shape))
print("Batch Target Masks: {}".format(batch_masks.shape))

In [ ]:
fm_shape=model.get_layer("Backbone").output[-1].shape[1::]
positional_encodings = create_positional_encodings(fm_shape, config.dim_transformer//2, batch_size=1)

print("Positional Encodings Shape: {}".format(positional_encodings.shape))


In [ ]:
for input in model.inputs:
    print("Expected Input: {}".format(input))

---

## 5. Inference

In [ ]:
%%time

if training_config["train_masks"]:
    detr_scores, detr_bbox, detr_masks = model([batch_images, positional_encodings], training=False)
else:
    detr_scores, detr_bbox = model([batch_images, positional_encodings], training=False)

print("Model Scores Shape: {}".format(detr_scores.shape))
print("Model Bounding Boxes Shape: {}".format(detr_bbox.shape))

---

## 6. Plott Results

In [ ]:
scores = detr_scores[0].numpy()
bboxs = detr_bbox[0].numpy()
image = batch_images[0].numpy()

In [ ]:
def rescale_bbox(bbox, height, width):
    x1, y1, w, h = bbox
    
    x1 = x1 * width
    y1 = y1 * height
    w = w * width
    h = h * height
    return (x1,y1,w,h)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
color = (1.0, 0.0, 0.0)

height, width = image.shape[:2]

for idx, score in enumerate(scores):
    
    if np.argmax(score) != 4:
        cls = np.argmax(score)
        conf = score[cls]
        print("Max Score: {} with Confidence {:.2f}".format(cls, conf*100))
        
        bbox = bboxs[idx, :]
        bbox = rescale_bbox(bbox, height, width)

        print("Bounding Box Coordinates: {}".format(bbox))
        
        apply_bbox(ax, bbox, color)

        
for bbox in batch_bboxs[0]:
    bbox = bbox.numpy()
    if (bbox == config.num_classes + 1).all():
        continue
    bbox = rescale_bbox(bbox, height, width)
    #apply_bbox(ax, bbox, color)
    
image = image.astype(np.uint8)
ax.imshow(image)
plt.show()

*This Notebook was created by: [auvisus GmbH](https://www.auvisus.com/)*